In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
from selenium import webdriver
import re
import json
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


## Parcing data

In [ ]:
URL = "https://everynoise.com/everydemo.cgi?vector=country&gscope=all&ascope=all"
page = requests.get(URL)

#print(page.text)

In [ ]:
soup = BeautifulSoup(page.content, "lxml")

In [ ]:
print(soup.tr)

In [ ]:
playlist_id = soup.tr.find_all("a", {"class": "note"})

id_list = []
for ele in playlist_id:
    s = str(ele)
    start = s.find("href=") + len("href=") + 1
    end = s.find("target=") - 2
    substring = s[start:end]
    id_list.append(substring)
    

len(id_list)

In [ ]:
s = '<a href="?root=AR%2055-64f&amp;gscope=all&amp;ascope=all" style="color: #779902" title="Re-sort the list by similarity to AR 55-64f.">Argentina</a>'
start = s.find(">") + 1
end = s.find("</a>")
substring = s[start:end]
print(substring)

In [ ]:
complete_info = soup.select("td > a:nth-of-type(1)")
#.find_all("td", {"class": "note"})

total_list = []
country_list = []
age_list = []
gender_list = []

for elem in complete_info:
    if (complete_info.index(elem) - 1)%4 == 0:
        country_list.append(str(elem))
    elif (complete_info.index(elem) - 2)%4 == 0:
        age_list.append(str(elem))
    elif (complete_info.index(elem) - 3)%4 == 0:
        gender_list.append(str(elem))

country_list

In [ ]:
def clean_list(lst):
    listed = []
    for e in lst:
        start = e.find(">") + 1
        end = e.find("</a>")
        substring = e[start:end]
        listed.append(substring)
    return listed

In [ ]:
country_list = clean_list(country_list)
age_list = clean_list(age_list)
gender_list = clean_list(gender_list)

In [ ]:
age_list

## Type of music for each demographic


### List of websites for each entry

In [ ]:

#sp = soup.tr.select('td.note,a.href')
cmpl = soup.find_all(href=re.compile("root"))

cmpl_list = []
for elem in cmpl:
    if (cmpl.index(elem) - 1)%3 == 0:
        cmpl_list.append(str(elem))

        
cmpl_list

In [ ]:
hlink_list = []
for s in cmpl_list:
    start = s.find("href=") + len("href=") + 1
    end = s.find("style=") - 2
    substring = s[start:end]
    hlink = "https://everynoise.com/everydemo.cgi" + substring
    hlink_list.append(hlink)

In [ ]:
hlink_list

In [ ]:
#for link in soup.tr.select('td.note,a.href').find_all('a'):
#    print(link.get('href'))

In [ ]:
df = pd.read_csv("scrape.csv")

In [ ]:
df["link"] = hlink_list

In [ ]:
df.to_csv("scrape.csv")

### List of music genres for each entry

In [ ]:
import re

In [ ]:
list_music = []
soup_list = []
for link in hlink_list:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")
    music = soup.find_all("div", {"class": "note"})
    soup_list.append(music)

soup_list


In [ ]:
soup_list = []
for link in hlink_list:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")
    
    soup_list.append(soup)

soup_list

In [ ]:
for e in soup_list:
    for entry in e:
        list_music.append(str((entry.get_text())))

In [ ]:
len(soup_list)

In [ ]:
list_music

In [ ]:
lst = []
for string in list_music:
    a = re.split(r"\n", string)[1:-1]
    lst.append(a)
lst

In [ ]:
df["music_genre"] = lst
df

In [ ]:
music = soup.find_all("div", {"class": "note"})

list_music = []
for e in music:
    list_music.append(str((e.get_text())))

list_music

In [ ]:
string = ['these people particularly like\nalbanian pop\nalbanian hip hop\nkosovan pop\n']

In [ ]:
string = ''.join(string)
string

In [ ]:
music_list = re.split(r"\n", string)[1:-1]
music_list

In [ ]:
info = soup.select("div a:nth-of-type(1)[href]")
info2 = soup.select("div > a")

In [ ]:
info

In [ ]:
for e in info:
    e.text.strip()

In [ ]:
info

## Obtain Spotify information

In [19]:
df = pd.read_csv("scrape.csv")

In [20]:
df

,Unnamed: 0,Country,Age,Gender,playlist_id,link,music_genre
0,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ..."
1,1,Albania,18-24,male,spotify:playlist:05tRphjiIyK7HDcZCCTgjs,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ..."
2,2,Algeria,0-17,male,spotify:playlist:0YUFKgR0FGGKd1zzO9XwEc,https://everynoise.com/everydemo.cgi?root=DZ%2...,"['rap algerien', 'rai', 'francoton', 'french h..."
3,3,Algeria,18-24,female,spotify:playlist:3gg3rmx4AadbMtpOVU2nvZ,https://everynoise.com/everydemo.cgi?root=DZ%2...,"['rai', 'moroccan pop', 'francoton', 'french h..."
4,4,Algeria,18-24,male,spotify:playlist:5P8wLuUHb6w5gV3ZAxLhft,https://everynoise.com/everydemo.cgi?root=DZ%2...,"['rai', 'rap algerien', 'rai algerien', 'moroc..."
...,...,...,...,...,...,...,...
1354,1354,Zambia,25-29,male,spotify:playlist:5oEDcbOlgT5ZD0wydiFgDY,https://everynoise.com/everydemo.cgi?root=ZM%2...,"['afropop', 'azontobeats', 'nigerian pop', 'af..."
1355,1355,Zambia,30-34,male,spotify:playlist:0zfoFflxig12fkZXw8Iope,https://everynoise.com/everydemo.cgi?root=ZM%2...,"['afropop', 'azontobeats', 'nigerian pop', 'af..."
1356,1356,Zimbabwe,18-24,female,spotify:playlist:4mlXOAZjVfrz3GKVyNmEjH,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',..."
1357,1357,Zimbabwe,18-24,male,spotify:playlist:22irF2EC5TL6lSYr6H9lni,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'amapiano', 'south african pop',..."


In [76]:
client_id = '8b66b90cecd04e10895f59bb12d19891'
client_secret = '4dee3f6c8d3d4537b304185074ceb8c5'

info = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager = info)



In [6]:
samp = 'spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw'

s = sp.playlist(samp)
artist_name = s["tracks"]["items"][0]["track"]["artists"][0]["name"]
artist_uri = s["tracks"]["items"][0]["track"]["artists"][0]["uri"]
track_id = s["tracks"]["items"][0]["track"]["uri"]
track_name = s["tracks"]["items"][0]["track"]["name"]
#["track"]["artists"]["uri"]

In [10]:
s["tracks"]["items"][0]["track"]["album"]["uri"]

'spotify:album:1zMIi85VGr1SMQQMKJj19J'

In [11]:
s["tracks"]["items"][1]["track"]["album"]["uri"]

'spotify:album:38fUKgktqd6THXBJtmUQTX'

In [25]:
samp = 'spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw'

s = sp.playlist(samp)
album_name = s["tracks"]["items"][0]["track"]["album"]["name"]
release_date = s["tracks"]["items"][0]["track"]["album"]["release_date"]
release_date_precision = s["tracks"]["items"][0]["track"]["album"]["release_date_precision"]
total_tracks =  s["tracks"]["items"][0]["track"]["album"]["total_tracks"]
uri_album = s["tracks"]["items"][0]["track"]["album"]["uri"]

#["track"]["artists"]["uri"]

In [12]:
len(s["tracks"]["items"])

42

In [31]:
empty = []
for i in range(len(s["tracks"]["items"])):
    dic = {"album_name": s["tracks"]["items"][i]["track"]["album"]["name"], 
            "release_date": s["tracks"]["items"][i]["track"]["album"]["release_date"], 
            "release_date_precision": s["tracks"]["items"][i]["track"]["album"]["release_date_precision"],
            "total_tracks": s["tracks"]["items"][i]["track"]["album"]["total_tracks"], 
            "uri_album": s["tracks"]["items"][i]["track"]["album"]["uri"]}
    empty.append(dic)

In [34]:
empty

[{'album_name': "Vjeshte / Dimer '18",
  'release_date': '2018-12-01',
  'release_date_precision': 'day',
  'total_tracks': 12,
  'uri_album': 'spotify:album:1zMIi85VGr1SMQQMKJj19J'},
 {'album_name': "Rapsodët n'rap T'sotit",
  'release_date': '2016-06-01',
  'release_date_precision': 'day',
  'total_tracks': 28,
  'uri_album': 'spotify:album:38fUKgktqd6THXBJtmUQTX'},
 {'album_name': 'UNITED STATE OF ALBANIA',
  'release_date': '2019-02-17',
  'release_date_precision': 'day',
  'total_tracks': 23,
  'uri_album': 'spotify:album:4bGvyFpxekT74M4b7zXh0X'},
 {'album_name': 'Pak Më Ndryshe',
  'release_date': '2015-08-06',
  'release_date_precision': 'day',
  'total_tracks': 13,
  'uri_album': 'spotify:album:5mmnzk08LTm4oWrEiTKZPU'},
 {'album_name': 'Papi Chulo',
  'release_date': '2022-06-15',
  'release_date_precision': 'day',
  'total_tracks': 1,
  'uri_album': 'spotify:album:2FJtK2m1duaeMTXp2V3ASG'},
 {'album_name': 'DAFINË MOJ',
  'release_date': '2021-10-01',
  'release_date_precision'

In [ ]:
for entry in df['playlist_id']:
    print(type(entry))

In [ ]:
test = df.head(10)

In [50]:
complete_list_info = []
def get_playlist_track(table):
    for entry in table['playlist_id']:
        s = sp.playlist(entry)
        each_list_info = []
        for i in range(len(s["tracks"]["items"])):
            dic = {"playlist_id": entry, 
                   "artist_name": s["tracks"]["items"][i]["track"]["artists"][0]["name"], 
                   "artist_uri": s["tracks"]["items"][i]["track"]["artists"][0]["uri"], 
                   "track_id": s["tracks"]["items"][i]["track"]["uri"], 
                   "track_name": s["tracks"]["items"][i]["track"]["name"], 
                   "album_name": s["tracks"]["items"][i]["track"]["album"]["name"], 
                   "release_date": s["tracks"]["items"][i]["track"]["album"]["release_date"], 
                   "release_date_precision": s["tracks"]["items"][i]["track"]["album"]["release_date_precision"], 
                   "total_tracks": s["tracks"]["items"][i]["track"]["album"]["total_tracks"], 
                   "uri_album": s["tracks"]["items"][i]["track"]["album"]["uri"]}
            each_list_info.append(dic)
        complete_list_info.append(each_list_info)
    return complete_list_info

In [47]:
album_list_info = []
def get_playlist_album(table):
    for entry in table['playlist_id']:
        s = sp.playlist(entry)
        each_album_info = []
        for i in range(len(s["tracks"]["items"])):
            dic = {"playlist_id": entry, 
                    "album_name": s["tracks"]["items"][i]["track"]["album"]["name"], 
                    "release_date": s["tracks"]["items"][i]["track"]["album"]["release_date"], 
                    "release_date_precision": s["tracks"]["items"][i]["track"]["album"]["release_date_precision"],
                    "total_tracks": s["tracks"]["items"][i]["track"]["album"]["total_tracks"], 
                    "uri_album": s["tracks"]["items"][i]["track"]["album"]["uri"]}
            each_album_info.append(dic)
        album_list_info.append(each_album_info)
    return album_list_info

In [51]:
df.head(10)

,Unnamed: 0,Country,Age,Gender,playlist_id,link,music_genre
0,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ..."
1,1,Albania,18-24,male,spotify:playlist:05tRphjiIyK7HDcZCCTgjs,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ..."
2,2,Algeria,0-17,male,spotify:playlist:0YUFKgR0FGGKd1zzO9XwEc,https://everynoise.com/everydemo.cgi?root=DZ%2...,"['rap algerien', 'rai', 'francoton', 'french h..."
3,3,Algeria,18-24,female,spotify:playlist:3gg3rmx4AadbMtpOVU2nvZ,https://everynoise.com/everydemo.cgi?root=DZ%2...,"['rai', 'moroccan pop', 'francoton', 'french h..."
4,4,Algeria,18-24,male,spotify:playlist:5P8wLuUHb6w5gV3ZAxLhft,https://everynoise.com/everydemo.cgi?root=DZ%2...,"['rai', 'rap algerien', 'rai algerien', 'moroc..."
5,5,Algeria,25-29,male,spotify:playlist:28hbIlbxxM0sOQVl2uZLBW,https://everynoise.com/everydemo.cgi?root=DZ%2...,"['rai', 'rap algerien', 'moroccan pop', 'rap m..."
6,6,Algeria,30-34,male,spotify:playlist:4eDfYwCZGDfifMN75BdnZU,https://everynoise.com/everydemo.cgi?root=DZ%2...,"['rai', 'moroccan pop', 'rap algerien', 'rap m..."
7,7,Angola,18-24,male,spotify:playlist:7rb1rciWmqeadwnAIp4mUk,https://everynoise.com/everydemo.cgi?root=AO%2...,"['rap angolano', 'hip hop tuga', 'kizomba', 't..."
8,8,Argentina,0-17,female,spotify:playlist:1KL27hViLJlh0fgawAU6SL,https://everynoise.com/everydemo.cgi?root=AR%2...,"['cumbia pop', 'trap argentino', 'rkt', 'cumbi..."
9,9,Argentina,0-17,male,spotify:playlist:2MpHlj7SuehxI7D56Tkg55,https://everynoise.com/everydemo.cgi?root=AR%2...,"['rkt', 'cumbia 420', 'trap argentino', 'cumbi..."


In [52]:
test = get_playlist_track(df.head(10))
test

[[{'playlist_id': 'spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw',
   'artist_name': 'Buta',
   'artist_uri': 'spotify:artist:1VZVyf06KS6zf8HpbW6WHd',
   'track_id': 'spotify:track:3erxsAHbtEbYWme16JOEoI',
   'track_name': 'My Bitch',
   'album_name': "Vjeshte / Dimer '18",
   'release_date': '2018-12-01',
   'release_date_precision': 'day',
   'total_tracks': 12,
   'uri_album': 'spotify:album:1zMIi85VGr1SMQQMKJj19J'},
  {'playlist_id': 'spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw',
   'artist_name': 'Mc Kresha',
   'artist_uri': 'spotify:artist:6fZ3U139Fb4VfOv7g63XhB',
   'track_id': 'spotify:track:2NHL2JXJ3li9fAlrZ3z4kO',
   'track_name': 'Era',
   'album_name': "Rapsodët n'rap T'sotit",
   'release_date': '2016-06-01',
   'release_date_precision': 'day',
   'total_tracks': 28,
   'uri_album': 'spotify:album:38fUKgktqd6THXBJtmUQTX'},
  {'playlist_id': 'spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw',
   'artist_name': 'Lyrical Son',
   'artist_uri': 'spotify:artist:5gfuF8sy6oqPt0162Z17Ru',
   'track

In [53]:
len(test)

10

In [56]:
compl_list = get_playlist_track(df)

In [62]:
len(compl_list)

1400

In [58]:
new_info = pd.DataFrame.from_dict(compl_list, orient='columns')

In [59]:
new_info

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,{'playlist_id': 'spotify:playlist:4Gol1EeoMD24...,...,None,None,None,None,None,None,None,None,None,None
1,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,{'playlist_id': 'spotify:playlist:05tRphjiIyK7...,...,None,None,None,None,None,None,None,None,None,None
2,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,{'playlist_id': 'spotify:playlist:0YUFKgR0FGGK...,...,None,None,None,None,None,None,None,None,None,None
3,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,{'playlist_id': 'spotify:playlist:3gg3rmx4Aadb...,...,None,None,None,None,None,None,None,None,None,None
4,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...,{'playlist_id': 'spotify:playlist:5P8wLuUHb6w5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,{'playlist_id': 'spotify:playlist:5oEDcbOlgT5Z...,...,None,None,None,None,None,None,None,None,None,None
1396,{'playlist_id': 'spotify:playlist:0zfoFflxig12...,{'playlist_id

In [ ]:
pd.Series

In [60]:
df2 = pd.DataFrame({"info": compl_list})
df2

,info
0,[{'playlist_id': 'spotify:playlist:4Gol1EeoMD2...
1,[{'playlist_id': 'spotify:playlist:05tRphjiIyK...
2,[{'playlist_id': 'spotify:playlist:0YUFKgR0FGG...
3,[{'playlist_id': 'spotify:playlist:3gg3rmx4Aad...
4,[{'playlist_id': 'spotify:playlist:5P8wLuUHb6w...
...,...
1395,[{'playlist_id': 'spotify:playlist:5oEDcbOlgT5...
1396,[{'playlist_id': 'spotify:playlist:0zfoFflxig1...
1397,[{'playlist_id': 'spotify:playlist:4mlXOAZjVfr...
1398,[{'playlist_id': 'spotify:playlist:22irF2EC5TL...


In [ ]:
df2.to_csv("playlist_info.csv")

In [63]:
test = df2.head(10)

In [64]:
df_list = []
for info in test["info"]:
    pl_df = pd.DataFrame(info)
    df_list.append(pl_df)
    new_df = pd.concat(df_list)
    
new_df  

,playlist_id,artist_name,artist_uri,track_id,track_name,album_name,release_date,release_date_precision,total_tracks,uri_album
0,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Buta,spotify:artist:1VZVyf06KS6zf8HpbW6WHd,spotify:track:3erxsAHbtEbYWme16JOEoI,My Bitch,Vjeshte / Dimer '18,2018-12-01,day,12,spotify:album:1zMIi85VGr1SMQQMKJj19J
1,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Mc Kresha,spotify:artist:6fZ3U139Fb4VfOv7g63XhB,spotify:track:2NHL2JXJ3li9fAlrZ3z4kO,Era,Rapsodët n'rap T'sotit,2016-06-01,day,28,spotify:album:38fUKgktqd6THXBJtmUQTX
2,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Lyrical Son,spotify:artist:5gfuF8sy6oqPt0162Z17Ru,spotify:track:0Gu5kKGMUuHgHk6KuP4i48,Menemadhe,UNITED STATE OF ALBANIA,2019-02-17,day,23,spotify:album:4bGvyFpxekT74M4b7zXh0X
3,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Noizy,spotify:artist:2SabhGDiBSyaM6QSuBUVqB,spotify:track:7za5XorLXqE1cWS2JFsB43,Otr Me Zemer,Pak Më Ndryshe,2015-08-06,day,13,spotify:album:5mmnzk08LTm4oWrEiTKZPU
4,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Matolale,spotify:artist:4GGoNiUyRvckodCeO3MZEG,spotify:track:4OMpyUdMdC7kGWT5iJ9X4a,Papi Chulo,Papi Chulo,2022-06-15,day,1,spotify:album:2FJtK2m1duaeMTXp2V3ASG
...,...,...,...,...,...,...,...,...,...,...
95,spotify:playlist:2MpHlj7SuehxI7D56Tkg55,Jona Caso,spotify:artist:4q5B9gAXSE4QLtQfqqRo5m,spotify:track:3BV3yf7oGFV64cvqjn9rOP,Darte - Turreo Edit,Darte (Turreo Edit),2022-03-25,day,1,spotify:album:1xB8ZyITW0QHkhgCu0gTGQ
96,spotify:playlist:2MpHlj7SuehxI7D56Tkg55,Godo,spotify:artist:7tfILFKScRBGGdo0GQkbB9,spotify:track:6F8y6NiaL6eLHyk9Sgi4tY,Diabla,Diabla,2019-06-22,day,1,spotify:album:4LCogn8xooemEe4j5rekgL
97,spotify:playlist:2MpHlj7SuehxI7D56Tkg55,Bhavi,spotify:artist:7fT2Me47PQ8T7954PKrcwR,spotify:track:2nxft8uvdFYovqAcaxulSz,Piso,Piso,2018-04-25,day,1,spotify:album:4Vsl4WJiebxjDwpBp7JqPQ
98,spotify:playlist:2MpHlj7SuehxI7D56Tkg55,Juani Dj,spotify:artist:422q9S13X3zRD8Bqbu7zXb,spotify:track:0drn6NOyot9jxOrKDIj9VA,Perreo Miky Woodz,Perreo Miky Woodz,2021-04-09,day,1,spotify:album:0C2kl6s36qG9vHYNTb2cOe


In [65]:
df_list = []
for info in df2["info"]:
    pl_df = pd.DataFrame(info)
    df_list.append(pl_df)
    new_df = pd.concat(df_list)
    
new_df  


,playlist_id,artist_name,artist_uri,track_id,track_name,album_name,release_date,release_date_precision,total_tracks,uri_album
0,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Buta,spotify:artist:1VZVyf06KS6zf8HpbW6WHd,spotify:track:3erxsAHbtEbYWme16JOEoI,My Bitch,Vjeshte / Dimer '18,2018-12-01,day,12,spotify:album:1zMIi85VGr1SMQQMKJj19J
1,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Mc Kresha,spotify:artist:6fZ3U139Fb4VfOv7g63XhB,spotify:track:2NHL2JXJ3li9fAlrZ3z4kO,Era,Rapsodët n'rap T'sotit,2016-06-01,day,28,spotify:album:38fUKgktqd6THXBJtmUQTX
2,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Lyrical Son,spotify:artist:5gfuF8sy6oqPt0162Z17Ru,spotify:track:0Gu5kKGMUuHgHk6KuP4i48,Menemadhe,UNITED STATE OF ALBANIA,2019-02-17,day,23,spotify:album:4bGvyFpxekT74M4b7zXh0X
3,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Noizy,spotify:artist:2SabhGDiBSyaM6QSuBUVqB,spotify:track:7za5XorLXqE1cWS2JFsB43,Otr Me Zemer,Pak Më Ndryshe,2015-08-06,day,13,spotify:album:5mmnzk08LTm4oWrEiTKZPU
4,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,Matolale,spotify:artist:4GGoNiUyRvckodCeO3MZEG,spotify:track:4OMpyUdMdC7kGWT5iJ9X4a,Papi Chulo,Papi Chulo,2022-06-15,day,1,spotify:album:2FJtK2m1duaeMTXp2V3ASG
...,...,...,...,...,...,...,...,...,...,...
28,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,Adekunle Gold,spotify:artist:2IK173RXLiCSQ8fhDlAb3s,spotify:track:7e9FvhmqibLIhZ6x8bKyT4,Sinner,Catch Me If You Can,2022-02-04,day,14,spotify:album:3b5r0ZryUlByiw2byA2IKn
29,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,Davido,spotify:artist:0Y3agQaa6g2r0YmHPOO9rh,spotify:track:3vTW0qXIitjhZW5kfTwBTv,Champion Sound,Champion Sound,2021-11-19,day,1,spotify:album:6pLdp90UOgmgb6yYh6bf2T
30,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,Burna Boy,spotify:artist:3wcj11K77LjEY1PkEazffa,spotify:track:0s5nhb6ts6uCKAVnGg46y6,Different Size (feat. Victony),"Love, Damini",2022-07-07,day,19,spotify:album:6kgDkAupBVRSqbJPUaTJwQ
31,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,Mayorkun,spotify:artist:3DNCUaKdMZcMVJIS7yTskd,spotify:track:6PojTTCNlz3qo3qBKuUyaO,Holy Father,Back In Office,2021-10-29,day,12,spotify:album:5vSyZglUnnQw4nlQC4QaRO


In [66]:
df2 = new_df

In [67]:
df2.to_csv("playlist_info_new.csv")

In [ ]:
df

In [68]:
result = pd.merge(df, df2, how="right", on=["playlist_id"])
result.rename(columns = {'Unnamed: 0': 'original_entry_index'}, inplace = True)


In [69]:
result

,original_entry_index,Country,Age,Gender,playlist_id,link,music_genre,artist_name,artist_uri,track_id,track_name,album_name,release_date,release_date_precision,total_tracks,uri_album
0,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Buta,spotify:artist:1VZVyf06KS6zf8HpbW6WHd,spotify:track:3erxsAHbtEbYWme16JOEoI,My Bitch,Vjeshte / Dimer '18,2018-12-01,day,12,spotify:album:1zMIi85VGr1SMQQMKJj19J
1,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Mc Kresha,spotify:artist:6fZ3U139Fb4VfOv7g63XhB,spotify:track:2NHL2JXJ3li9fAlrZ3z4kO,Era,Rapsodët n'rap T'sotit,2016-06-01,day,28,spotify:album:38fUKgktqd6THXBJtmUQTX
2,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Lyrical Son,spotify:artist:5gfuF8sy6oqPt0162Z17Ru,spotify:track:0Gu5kKGMUuHgHk6KuP4i48,Menemadhe,UNITED STATE OF ALBANIA,2019-02-17,day,23,spotify:album:4bGvyFpxekT74M4b7zXh0X
3,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Noizy,spotify:artist:2SabhGDiBSyaM6QSuBUVqB,spotify:track:7za5XorLXqE1cWS2JFsB43,Otr Me Zemer,Pak Më Ndryshe,2015-08-06,day,13,spotify:album:5mmnzk08LTm4oWrEiTKZPU
4,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Matolale,spotify:artist:4GGoNiUyRvckodCeO3MZEG,spotify:track:4OMpyUdMdC7kGWT5iJ9X4a,Papi Chulo,Papi Chulo,2022-06-15,day,1,spotify:album:2FJtK2m1duaeMTXp2V3ASG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126366,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Adekunle Gold,spotify:artist:2IK173RXLiCSQ8fhDlAb3s,spotify:track:7e9FvhmqibLIhZ6x8bKyT4,Sinner,Catch Me If You Can,2022-02-04,day,14,spotify:album:3b5r0ZryUlByiw2byA2IKn
126367,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Davido,spotify:artist:0Y3agQaa6g2r0YmHPOO9rh,spotify:track:3vTW0qXIitjhZW5kfTwBTv,Champion Sound,Champion Sound,2021-11-19,day,1,spotify:album:6pLdp90UOgmgb6yYh6bf2T
126368,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Burna Boy,spotify:artist:3wcj11K77LjEY1PkEazffa,spotify:track:0s5nhb6ts6uCKAVnGg46y6,Different Size (feat. Victony),"Love, Damini",2022-07-07,day,19,spotify:album:6kgDkAupBVRSqbJPUaTJwQ
126369,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Mayorkun,spotify:artist:3DNCUaKdMZcMVJIS7yTskd,spotify:track:6PojTTCNlz3qo3qBKuUyaO,Holy Father,Back In Office,2021-10-29,day,12,spotify:album:5vSyZglUnnQw4nlQC4QaRO


In [70]:
result.to_csv("playlist_info_new.csv")

In [ ]:
sample_comp = get_playlist_track(test)

## Obtain spotify features

In [71]:
import numpy as np
import pandas as pd

In [72]:
results = pd.read_csv("playlist_info_new.csv")
results

,Unnamed: 0,original_entry_index,Country,Age,Gender,playlist_id,link,music_genre,artist_name,artist_uri,track_id,track_name,album_name,release_date,release_date_precision,total_tracks,uri_album
0,0,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Buta,spotify:artist:1VZVyf06KS6zf8HpbW6WHd,spotify:track:3erxsAHbtEbYWme16JOEoI,My Bitch,Vjeshte / Dimer '18,2018-12-01,day,12,spotify:album:1zMIi85VGr1SMQQMKJj19J
1,1,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Mc Kresha,spotify:artist:6fZ3U139Fb4VfOv7g63XhB,spotify:track:2NHL2JXJ3li9fAlrZ3z4kO,Era,Rapsodët n'rap T'sotit,2016-06-01,day,28,spotify:album:38fUKgktqd6THXBJtmUQTX
2,2,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Lyrical Son,spotify:artist:5gfuF8sy6oqPt0162Z17Ru,spotify:track:0Gu5kKGMUuHgHk6KuP4i48,Menemadhe,UNITED STATE OF ALBANIA,2019-02-17,day,23,spotify:album:4bGvyFpxekT74M4b7zXh0X
3,3,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Noizy,spotify:artist:2SabhGDiBSyaM6QSuBUVqB,spotify:track:7za5XorLXqE1cWS2JFsB43,Otr Me Zemer,Pak Më Ndryshe,2015-08-06,day,13,spotify:album:5mmnzk08LTm4oWrEiTKZPU
4,4,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Matolale,spotify:artist:4GGoNiUyRvckodCeO3MZEG,spotify:track:4OMpyUdMdC7kGWT5iJ9X4a,Papi Chulo,Papi Chulo,2022-06-15,day,1,spotify:album:2FJtK2m1duaeMTXp2V3ASG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126366,126366,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Adekunle Gold,spotify:artist:2IK173RXLiCSQ8fhDlAb3s,spotify:track:7e9FvhmqibLIhZ6x8bKyT4,Sinner,Catch Me If You Can,2022-02-04,day,14,spotify:album:3b5r0ZryUlByiw2byA2IKn
126367,126367,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Davido,spotify:artist:0Y3agQaa6g2r0YmHPOO9rh,spotify:track:3vTW0qXIitjhZW5kfTwBTv,Champion Sound,Champion Sound,2021-11-19,day,1,spotify:album:6pLdp90UOgmgb6yYh6bf2T
126368,126368,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Burna Boy,spotify:artist:3wcj11K77LjEY1PkEazffa,spotify:track:0s5nhb6ts6uCKAVnGg46y6,Different Size (feat. Victony),"Love, Damini",2022-07-07,day,19,spotify:album:6kgDkAupBVRSqbJPUaTJwQ
126369,126369,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Mayorkun,spotify:artist:3DNCUaKdMZcMVJIS7yTskd,spotify:track:6PojTTCNlz3qo3qBKuUyaO,Holy Father,Back In Office,2021-10-29,day,12,spotify:album:5vSyZglUnnQw4nlQC4QaRO


In [73]:
list_track = results["track_id"].values.tolist()


In [74]:
results["track_id"].isnull().values.any()

False

In [53]:
x = range(6)
for n in x:
  print(n)

0
1
2
3
4
5


In [77]:
gp_empty = []
for k in range(1263):
    gp = sp.audio_features(list_track[k*100:(k+1)*100])
    gp_empty.append(gp)

In [78]:
from itertools import chain
newlist = list(chain(*gp_empty))
len(newlist)
rest_list = sp.audio_features(list_track[126300:])
len(rest_list)
audio = newlist + rest_list
audio

[{'danceability': 0.799,
  'energy': 0.569,
  'key': 11,
  'loudness': -7.94,
  'mode': 0,
  'speechiness': 0.238,
  'acousticness': 0.521,
  'instrumentalness': 0,
  'liveness': 0.121,
  'valence': 0.567,
  'tempo': 82.962,
  'type': 'audio_features',
  'id': '3erxsAHbtEbYWme16JOEoI',
  'uri': 'spotify:track:3erxsAHbtEbYWme16JOEoI',
  'track_href': 'https://api.spotify.com/v1/tracks/3erxsAHbtEbYWme16JOEoI',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3erxsAHbtEbYWme16JOEoI',
  'duration_ms': 187139,
  'time_signature': 4},
 {'danceability': 0.695,
  'energy': 0.772,
  'key': 2,
  'loudness': -5.675,
  'mode': 1,
  'speechiness': 0.327,
  'acousticness': 0.0176,
  'instrumentalness': 0.00884,
  'liveness': 0.316,
  'valence': 0.35,
  'tempo': 150.035,
  'type': 'audio_features',
  'id': '2NHL2JXJ3li9fAlrZ3z4kO',
  'uri': 'spotify:track:2NHL2JXJ3li9fAlrZ3z4kO',
  'track_href': 'https://api.spotify.com/v1/tracks/2NHL2JXJ3li9fAlrZ3z4kO',
  'analysis_url': 'https://api.spo

In [79]:
len(audio)

126371

In [80]:
audio_copy = audio.copy()
len(audio_copy)

126371

In [90]:
results["audio_features"] = audio_copy
results
results.to_csv("playlist_info_new.csv")

In [91]:

results = pd.read_csv("playlist_info_new.csv")
results['audio_features'][9]

"{'danceability': 0.809, 'energy': 0.534, 'key': 7, 'loudness': -7.439, 'mode': 1, 'speechiness': 0.273, 'acousticness': 0.016, 'instrumentalness': 0, 'liveness': 0.184, 'valence': 0.384, 'tempo': 84.888, 'type': 'audio_features', 'id': '1vorfhMyr6CJmzGZgyLigc', 'uri': 'spotify:track:1vorfhMyr6CJmzGZgyLigc', 'track_href': 'https://api.spotify.com/v1/tracks/1vorfhMyr6CJmzGZgyLigc', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1vorfhMyr6CJmzGZgyLigc', 'duration_ms': 163765, 'time_signature': 4}"

In [92]:
Nan = "{'danceability': 0,'energy': 0,'key': 0,'loudness': 0,'mode': 0,'speechiness': 0,'acousticness': 0,'instrumentalness': 0,'liveness': 0,'valence': 0,'tempo': 0,'type': 'audio_features','id': '','uri': '','track_href': '','analysis_url': '','duration_ms': 0,'time_signature': 0}"

In [93]:
res = results['audio_features'].replace(np.nan, Nan)
res.isnull().values.any()

False

In [94]:
audio_list = res.values.tolist()
audio_list

["{'danceability': 0.799, 'energy': 0.569, 'key': 11, 'loudness': -7.94, 'mode': 0, 'speechiness': 0.238, 'acousticness': 0.521, 'instrumentalness': 0, 'liveness': 0.121, 'valence': 0.567, 'tempo': 82.962, 'type': 'audio_features', 'id': '3erxsAHbtEbYWme16JOEoI', 'uri': 'spotify:track:3erxsAHbtEbYWme16JOEoI', 'track_href': 'https://api.spotify.com/v1/tracks/3erxsAHbtEbYWme16JOEoI', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3erxsAHbtEbYWme16JOEoI', 'duration_ms': 187139, 'time_signature': 4}",
 "{'danceability': 0.695, 'energy': 0.772, 'key': 2, 'loudness': -5.675, 'mode': 1, 'speechiness': 0.327, 'acousticness': 0.0176, 'instrumentalness': 0.00884, 'liveness': 0.316, 'valence': 0.35, 'tempo': 150.035, 'type': 'audio_features', 'id': '2NHL2JXJ3li9fAlrZ3z4kO', 'uri': 'spotify:track:2NHL2JXJ3li9fAlrZ3z4kO', 'track_href': 'https://api.spotify.com/v1/tracks/2NHL2JXJ3li9fAlrZ3z4kO', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2NHL2JXJ3li9fAlrZ3z4kO', 'duration

In [129]:
for idx, x in enumerate(results["audio_features"]):
    if x is None:
        x.repalce(Nan)
    else:
        pass

In [111]:
results['audio_features'] = np.where(results['audio_features'] is None, Nan, results['audio_features'])

In [130]:
audio_list = results['audio_features'].values.tolist()
len(audio_list)

123025

In [95]:
#convert string dictionary to dictionary
import ast
audio_diclist = []
for ele in audio_list:
    resu = ast.literal_eval(ele)
    audio_diclist.append(resu)
    
audio_diclist

[{'danceability': 0.799,
  'energy': 0.569,
  'key': 11,
  'loudness': -7.94,
  'mode': 0,
  'speechiness': 0.238,
  'acousticness': 0.521,
  'instrumentalness': 0,
  'liveness': 0.121,
  'valence': 0.567,
  'tempo': 82.962,
  'type': 'audio_features',
  'id': '3erxsAHbtEbYWme16JOEoI',
  'uri': 'spotify:track:3erxsAHbtEbYWme16JOEoI',
  'track_href': 'https://api.spotify.com/v1/tracks/3erxsAHbtEbYWme16JOEoI',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3erxsAHbtEbYWme16JOEoI',
  'duration_ms': 187139,
  'time_signature': 4},
 {'danceability': 0.695,
  'energy': 0.772,
  'key': 2,
  'loudness': -5.675,
  'mode': 1,
  'speechiness': 0.327,
  'acousticness': 0.0176,
  'instrumentalness': 0.00884,
  'liveness': 0.316,
  'valence': 0.35,
  'tempo': 150.035,
  'type': 'audio_features',
  'id': '2NHL2JXJ3li9fAlrZ3z4kO',
  'uri': 'spotify:track:2NHL2JXJ3li9fAlrZ3z4kO',
  'track_href': 'https://api.spotify.com/v1/tracks/2NHL2JXJ3li9fAlrZ3z4kO',
  'analysis_url': 'https://api.spo

In [96]:
audio_df = pd.DataFrame(audio_diclist)
audio_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.799,0.569,11,-7.940,0,0.2380,0.52100,0.000000,0.1210,0.567,82.962,audio_features,3erxsAHbtEbYWme16JOEoI,spotify:track:3erxsAHbtEbYWme16JOEoI,https://api.spotify.com/v1/tracks/3erxsAHbtEbY...,https://api.spotify.com/v1/audio-analysis/3erx...,187139,4
1,0.695,0.772,2,-5.675,1,0.3270,0.01760,0.008840,0.3160,0.350,150.035,audio_features,2NHL2JXJ3li9fAlrZ3z4kO,spotify:track:2NHL2JXJ3li9fAlrZ3z4kO,https://api.spotify.com/v1/tracks/2NHL2JXJ3li9...,https://api.spotify.com/v1/audio-analysis/2NHL...,246608,4
2,0.856,0.683,4,-4.645,0,0.1180,0.00908,0.000301,0.0726,0.636,98.021,audio_features,0Gu5kKGMUuHgHk6KuP4i48,spotify:track:0Gu5kKGMUuHgHk6KuP4i48,https://api.spotify.com/v1/tracks/0Gu5kKGMUuHg...,https://api.spotify.com/v1/audio-analysis/0Gu5...,193776,4
3,0.583,0.796,4,-7.885,0,0.0748,0.02280,0.000000,0.1880,0.611,94.986,audio_features,7za5XorLXqE1cWS2JFsB43,spotify:track:7za5XorLXqE1cWS2JFsB43,https://api.spotify.com/v1/tracks/7za5XorLXqE1...,https://api.spotify.com/v1/audio-analysis/7za5...,248333,4
4,0.875,0.636,2,-7.220,1,0.1290,0.06860,0.000009,0.0677,0.428,95.031,audio_features,4OMpyUdMdC7kGWT5iJ9X4a,spotify:track:4OMpyUdMdC7kGWT5iJ9X4a,https://api.spotify.com/v1/tracks/4OMpyUdMdC7k...,https://api.spotify.com/v1/audio-analysis/4OMp...,148870,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126366,0.505,0.548,1,-6.508,0,0.3980,0.75700,0.000000,0.1980,0.515,136.352,audio_features,7e9FvhmqibLIhZ6x8bKyT4,spotify:track:7e9FvhmqibLIhZ6x8bKyT4,https://api.spotify.com/v1/tracks/7e9FvhmqibLI...,https://api.spotify.com/v1/audio-analysis/7e9F...,174118,5
126367,0.861,0.475,1,-11.952,0,0.0537,0.03250,0.322000,0.0708,0.248,113.016,audio_features,3vTW0qXIitjhZW5kfTwBTv,spotify:track:3vTW0qXIitjhZW5kfTwBTv,https://api.spotify.com/v1/tracks/3vTW0qXIitjh...,https://api.spotify.com/v1/audio-analysis/3vTW...,237876,4
126368,0.731,0.755,6,-9.096,1,0.0359,0.60000,0.013400,0.1910,0.838,107.973,audio_features,0s5nhb6ts6uCKAVnGg46y6,spotify:track:0s5nhb6ts6uCKAVnGg46y6,https://api.spotify.com/v1/tracks/0s5nhb6ts6uC...,https://api.spotify.com/v1/audio-analysis/0s5n...,209969,4
126369,0.784,0.865,11,-5.679,0,0.1550,0.30100,0.000324,0.1100,0.491,115.960,audio_features,6PojTTCNlz3qo3qBKuUyaO,spotify:track:6PojTTCNlz3qo3qBKuUyaO,https://api.spotify.com/v1/tracks/6PojTTCNlz3q...,https://api.spotify.com/v1/audio-analysis/6Poj...,184000,4


In [97]:
FINAL = pd.concat([results, audio_df], axis=1, join="inner")

FINAL


,Unnamed: 0.1,Unnamed: 0,original_entry_index,Country,Age,Gender,playlist_id,link,music_genre,artist_name,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,0,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Buta,...,0.1210,0.567,82.962,audio_features,3erxsAHbtEbYWme16JOEoI,spotify:track:3erxsAHbtEbYWme16JOEoI,https://api.spotify.com/v1/tracks/3erxsAHbtEbY...,https://api.spotify.com/v1/audio-analysis/3erx...,187139,4
1,1,1,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Mc Kresha,...,0.3160,0.350,150.035,audio_features,2NHL2JXJ3li9fAlrZ3z4kO,spotify:track:2NHL2JXJ3li9fAlrZ3z4kO,https://api.spotify.com/v1/tracks/2NHL2JXJ3li9...,https://api.spotify.com/v1/audio-analysis/2NHL...,246608,4
2,2,2,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Lyrical Son,...,0.0726,0.636,98.021,audio_features,0Gu5kKGMUuHgHk6KuP4i48,spotify:track:0Gu5kKGMUuHgHk6KuP4i48,https://api.spotify.com/v1/tracks/0Gu5kKGMUuHg...,https://api.spotify.com/v1/audio-analysis/0Gu5...,193776,4
3,3,3,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Noizy,...,0.1880,0.611,94.986,audio_features,7za5XorLXqE1cWS2JFsB43,spotify:track:7za5XorLXqE1cWS2JFsB43,https://api.spotify.com/v1/tracks/7za5XorLXqE1...,https://api.spotify.com/v1/audio-analysis/7za5...,248333,4
4,4,4,0,Albania,18-24,female,spotify:playlist:4Gol1EeoMD24Bm4EoiVpnw,https://everynoise.com/everydemo.cgi?root=AL%2...,"['albanian pop', 'albanian hip hop', 'kosovan ...",Matolale,...,0.0677,0.428,95.031,audio_features,4OMpyUdMdC7kGWT5iJ9X4a,spotify:track:4OMpyUdMdC7kGWT5iJ9X4a,https://api.spotify.com/v1/tracks/4OMpyUdMdC7k...,https://api.spotify.com/v1/audio-analysis/4OMp...,148870,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126366,126366,126366,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Adekunle Gold,...,0.1980,0.515,136.352,audio_features,7e9FvhmqibLIhZ6x8bKyT4,spotify:track:7e9FvhmqibLIhZ6x8bKyT4,https://api.spotify.com/v1/tracks/7e9FvhmqibLI...,https://api.spotify.com/v1/audio-analysis/7e9F...,174118,5
126367,126367,126367,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Davido,...,0.0708,0.248,113.016,audio_features,3vTW0qXIitjhZW5kfTwBTv,spotify:track:3vTW0qXIitjhZW5kfTwBTv,https://api.spotify.com/v1/tracks/3vTW0qXIitjh...,https://api.spotify.com/v1/audio-analysis/3vTW...,237876,4
126368,126368,126368,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Burna Boy,...,0.1910,0.838,107.973,audio_features,0s5nhb6ts6uCKAVnGg46y6,spotify:track:0s5nhb6ts6uCKAVnGg46y6,https://api.spotify.com/v1/tracks/0s5nhb6ts6uC...,https://api.spotify.com/v1/audio-analysis/0s5n...,209969,4
126369,126369,126369,1358,Zimbabwe,25-29,male,spotify:playlist:6seoeGVArUcOFrKXAmLnqp,https://everynoise.com/everydemo.cgi?root=ZW%2...,"['afro soul', 'south african pop', 'amapiano',...",Mayorkun,...,0.1100,0.491,115.960,audio_features,6PojTTCNlz3qo3qBKuUyaO,spotify:track:6PojTTCNlz3qo3qBKuUyaO,https://api.spotify.com/v1/tracks/6PojTTCNlz3q...,https://api.spotify.com/v1/audio-analysis/6Poj...,184000,4


In [98]:
FINAL = pd.concat([results, audio_df], axis=1, join="inner")

FINAL.rename({'Unnamed: 0': 'index'}, axis=1, inplace=True)
F= FINAL.drop(columns = ['Unnamed: 0.1'])

In [99]:
F.to_csv("playlist_info_new.csv")

In [ ]:
#NOT USED

In [ ]:
sp.audio_features(list_track[0:100])

In [38]:
df_list_sample = []
for track in list_track[:10]:
    client_id = '8b66b90cecd04e10895f59bb12d19891'
    client_secret = '4dee3f6c8d3d4537b304185074ceb8c5'
    info = SpotifyClientCredentials(client_id, client_secret)
    sp = spotipy.Spotify(client_credentials_manager = info)
    features = sp.audio_features(track)
    features_df = pd.DataFrame(data=features)
    df_list_sample.append(features_df)

In [39]:
df_list_sample

[   danceability  energy  key  loudness  mode  speechiness  acousticness  \
 0         0.637   0.806   10    -4.929     0        0.112         0.391   
 
    instrumentalness  liveness  valence    tempo            type  \
 0          0.000386     0.126    0.722  198.073  audio_features   
 
                        id                                   uri  \
 0  5JAVrk1CBKoS3LE0xsEd4G  spotify:track:5JAVrk1CBKoS3LE0xsEd4G   
 
                                           track_href  \
 0  https://api.spotify.com/v1/tracks/5JAVrk1CBKoS...   
 
                                         analysis_url  duration_ms  \
 0  https://api.spotify.com/v1/audio-analysis/5JAV...       168598   
 
    time_signature  
 0               4  ,
    danceability  energy  key  loudness  mode  speechiness  acousticness  \
 0         0.744   0.676    6    -7.252     0        0.327         0.105   
 
    instrumentalness  liveness  valence   tempo            type  \
 0                 0      0.34    0.614  94.937 

In [26]:
merged = pd.concat(df_list_sample)
merged

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.637,0.806,10,-4.929,0,0.1120,0.3910,0.000386,0.1260,0.722,198.073,audio_features,5JAVrk1CBKoS3LE0xsEd4G,spotify:track:5JAVrk1CBKoS3LE0xsEd4G,https://api.spotify.com/v1/tracks/5JAVrk1CBKoS...,https://api.spotify.com/v1/audio-analysis/5JAV...,168598,4
0,0.744,0.676,6,-7.252,0,0.3270,0.1050,0.000000,0.3400,0.614,94.937,audio_features,4pkhngbmPDKrcZ0EVueSDO,spotify:track:4pkhngbmPDKrcZ0EVueSDO,https://api.spotify.com/v1/tracks/4pkhngbmPDKr...,https://api.spotify.com/v1/audio-analysis/4pkh...,197114,4
0,0.875,0.636,2,-7.220,1,0.1290,0.0686,0.000009,0.0677,0.428,95.031,audio_features,4OMpyUdMdC7kGWT5iJ9X4a,spotify:track:4OMpyUdMdC7kGWT5iJ9X4a,https://api.spotify.com/v1/tracks/4OMpyUdMdC7k...,https://api.spotify.com/v1/audio-analysis/4OMp...,148870,4
0,0.744,0.701,11,-5.619,1,0.1620,0.3040,0.000000,0.2490,0.516,178.005,audio_features,1rwCOEQKudAW1jaK235YBY,spotify:track:1rwCOEQKudAW1jaK235YBY,https://api.spotify.com/v1/tracks/1rwCOEQKudAW...,https://api.spotify.com/v1/audio-analysis/1rwC...,210337,4
0,0.718,0.808,5,-3.425,0,0.0655,0.1960,0.000000,0.1100,0.613,159.045,audio_features,7zbuyLlYiYyq7vIHyXdgKI,spotify:track:7zbuyLlYiYyq7vIHyXdgKI,https://api.spotify.com/v1/tracks/7zbuyLlYiYyq...,https://api.spotify.com/v1/audio-analysis/7zbu...,172639,4
0,0.825,0.433,9,-5.380,0,0.0377,0.5840,0.000000,0.0876,0.738,95.863,audio_features,2EQEVg53ESnhgPMOxIlNgl,spotify:track:2EQEVg53ESnhgPMOxIlNgl,https://api.spotify.com/v1/tracks/2EQEVg53ESnh...,https://api.spotify.com/v1/audio-analysis/2EQE...,183353,4
0,0.915,0.715,6,-6.738,0,0.0998,0.4600,0.000000,0.0856,0.841,124.952,audio_features,4PjFP7cs3tvNzZhZi7NCD9,spotify:track:4PjFP7cs3tvNzZhZi7NCD9,https://api.spotify.com/v1/tracks/4PjFP7cs3tvN...,https://api.spotify.com/v1/audio-analysis/4PjF...,190080,4
0,0.815,0.947,1,-3.022,0,0.0536,0.3110,0.179000,0.1650,0.664,96.004,audio_features,2mJ3U7zb0Ew7iw2ukeGkev,spotify:track:2mJ3U7zb0Ew7iw2ukeGkev,https://api.spotify.com/v1/tracks/2mJ3U7zb0Ew7...,https://api.spotify.com/v1/audio-analysis/2mJ3...,165462,4
0,0.684,0.707,3,-4.851,0,0.0926,0.1370,0.000002,0.1370,0.725,81.689,audio_features,5M5T9rswunAx6vX8Odd3HD,spotify:track:5M5T9rswunAx6vX8Odd3HD,https://api.spotify.com/v1/tracks/5M5T9rswunAx...,https://api.spotify.com/v1/audio-analysis/5M5T...,143099,4
0,0.817,0.597,2,-6.551,0,0.0634,0.4570,0.000023,0.0858,0.426,97.077,audio_features,0j3qomI0ce8LdPJvhTo6VU,spotify:track:0j3qomI0ce8LdPJvhTo6VU,https://api.spotify.com/v1/tracks/0j3qomI0ce8L...,https://api.spotify.com/v1/audio-analysis/0j3q...,205361,4


In [32]:
df_list = []
for track in list_track:
    client_id = '8b66b90cecd04e10895f59bb12d19891'
    client_secret = '4dee3f6c8d3d4537b304185074ceb8c5'
    info = SpotifyClientCredentials(client_id, client_secret)
    sp = spotipy.Spotify(client_credentials_manager = info)
    features = sp.audio_features(track)
    features_df = pd.DataFrame(data=features)
    df_list.append(features_df)

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [15]:
results_copy = results.copy().head(10)

In [19]:
df_list[0]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.637,0.806,10,-4.929,0,0.112,0.391,0.000386,0.126,0.722,198.073,audio_features,5JAVrk1CBKoS3LE0xsEd4G,spotify:track:5JAVrk1CBKoS3LE0xsEd4G,https://api.spotify.com/v1/tracks/5JAVrk1CBKoS...,https://api.spotify.com/v1/audio-analysis/5JAV...,168598,4


In [20]:
merged = pd.concat(df_list)
merged

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.637,0.806,10,-4.929,0,0.1120,0.3910,0.000386,0.1260,0.722,198.073,audio_features,5JAVrk1CBKoS3LE0xsEd4G,spotify:track:5JAVrk1CBKoS3LE0xsEd4G,https://api.spotify.com/v1/tracks/5JAVrk1CBKoS...,https://api.spotify.com/v1/audio-analysis/5JAV...,168598,4
0,0.744,0.676,6,-7.252,0,0.3270,0.1050,0.000000,0.3400,0.614,94.937,audio_features,4pkhngbmPDKrcZ0EVueSDO,spotify:track:4pkhngbmPDKrcZ0EVueSDO,https://api.spotify.com/v1/tracks/4pkhngbmPDKr...,https://api.spotify.com/v1/audio-analysis/4pkh...,197114,4
0,0.875,0.636,2,-7.220,1,0.1290,0.0686,0.000009,0.0677,0.428,95.031,audio_features,4OMpyUdMdC7kGWT5iJ9X4a,spotify:track:4OMpyUdMdC7kGWT5iJ9X4a,https://api.spotify.com/v1/tracks/4OMpyUdMdC7k...,https://api.spotify.com/v1/audio-analysis/4OMp...,148870,4
0,0.744,0.701,11,-5.619,1,0.1620,0.3040,0.000000,0.2490,0.516,178.005,audio_features,1rwCOEQKudAW1jaK235YBY,spotify:track:1rwCOEQKudAW1jaK235YBY,https://api.spotify.com/v1/tracks/1rwCOEQKudAW...,https://api.spotify.com/v1/audio-analysis/1rwC...,210337,4
0,0.718,0.808,5,-3.425,0,0.0655,0.1960,0.000000,0.1100,0.613,159.045,audio_features,7zbuyLlYiYyq7vIHyXdgKI,spotify:track:7zbuyLlYiYyq7vIHyXdgKI,https://api.spotify.com/v1/tracks/7zbuyLlYiYyq...,https://api.spotify.com/v1/audio-analysis/7zbu...,172639,4
0,0.825,0.433,9,-5.380,0,0.0377,0.5840,0.000000,0.0876,0.738,95.863,audio_features,2EQEVg53ESnhgPMOxIlNgl,spotify:track:2EQEVg53ESnhgPMOxIlNgl,https://api.spotify.com/v1/tracks/2EQEVg53ESnh...,https://api.spotify.com/v1/audio-analysis/2EQE...,183353,4
0,0.915,0.715,6,-6.738,0,0.0998,0.4600,0.000000,0.0856,0.841,124.952,audio_features,4PjFP7cs3tvNzZhZi7NCD9,spotify:track:4PjFP7cs3tvNzZhZi7NCD9,https://api.spotify.com/v1/tracks/4PjFP7cs3tvN...,https://api.spotify.com/v1/audio-analysis/4PjF...,190080,4
0,0.815,0.947,1,-3.022,0,0.0536,0.3110,0.179000,0.1650,0.664,96.004,audio_features,2mJ3U7zb0Ew7iw2ukeGkev,spotify:track:2mJ3U7zb0Ew7iw2ukeGkev,https://api.spotify.com/v1/tracks/2mJ3U7zb0Ew7...,https://api.spotify.com/v1/audio-analysis/2mJ3...,165462,4
0,0.684,0.707,3,-4.851,0,0.0926,0.1370,0.000002,0.1370,0.725,81.689,audio_features,5M5T9rswunAx6vX8Odd3HD,spotify:track:5M5T9rswunAx6vX8Odd3HD,https://api.spotify.com/v1/tracks/5M5T9rswunAx...,https://api.spotify.com/v1/audio-analysis/5M5T...,143099,4
0,0.817,0.597,2,-6.551,0,0.0634,0.4570,0.000023,0.0858,0.426,97.077,audio_features,0j3qomI0ce8LdPJvhTo6VU,spotify:track:0j3qomI0ce8LdPJvhTo6VU,https://api.spotify.com/v1/tracks/0j3qomI0ce8L...,https://api.spotify.com/v1/audio-analysis/0j3q...,205361,4


In [16]:
results_copy["audio"] = df_list

In [18]:
results_copy["audio"][0]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.637,0.806,10,-4.929,0,0.112,0.391,0.000386,0.126,0.722,198.073,audio_features,5JAVrk1CBKoS3LE0xsEd4G,spotify:track:5JAVrk1CBKoS3LE0xsEd4G,https://api.spotify.com/v1/tracks/5JAVrk1CBKoS...,https://api.spotify.com/v1/audio-analysis/5JAV...,168598,4


In [18]:
list_track = results["track_id"].values.tolist()

df_list = []
for track in list_track:
    features = sp.audio_features(track)
    features_df = pd.DataFrame(data=features)
    df_list.append(features_df)

#    for i in range(len(list_track)):
#        print(features)
#features_df = pd.DataFrame(data=features, columns=features[0].keys())
#features

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Scrape Song Lyrics

## Create DataFrame/'


In [ ]:
# List1 
lst = [country_list, age_list, gender_list, id_list]
    
df = pd.DataFrame(lst)

In [ ]:
df = df.transpose()

In [ ]:
df = df.rename(columns={0: "Country", 1: "Age", 2: "Gender", 3: "playlist_id"})

In [ ]:
df = df.drop('Unnamed: 0', axis=1)
df

In [ ]:
df.to_csv("scrape.csv")